In [ ]:
class Graph:
  def __init__(self):
    self.graph = {} #lista de adyacencia
  
  def add_vertex(self, label):
    if label not in self.graph.keys():
      self.graph[label] = [] #nodo desconectado

  def has_edge(self, v1, v2):
    return v2 in self.graph[v1]

  def add_edge(self, v1, v2, directed = True):
    self.add_vertex(v1)
    self.add_vertex(v2)

    if(not directed and not self.has_edge(v2,v1)):
        self.graph[v2].append(v1)
        return
    if(not self.has_edge(v1,v2)):
      self.graph[v1].append(v2)
    if(not self.has_edge(v2,v1)):
      self.graph[v2].append(v1)

  def print_graph(self):
    print(self.graph)

  def DFS(self, u, vis=[]):
    if u not in vis: 
      vis.append(u) 
      for i in self.graph[u]: 
        if i not in vis: 
          vis.append(i) 
          self.DFS(i, vis) 
      return vis

def limpiar_diccionario(dictio:dict):#Funcion creada para eliminar del diccionario elementos conflictivos que dañen la ejecucion del programa
  toTakeAout=[]#lista de las llaves a eliminar
  for key in dictio:#se verifica que el valor guardado en el dict no sea nulo
    if not dictio[key]:
      toTakeAout.append(key)
    elif not dictio[key][0]:#se verifica que el valor guardado en la primera posicion de la lista que guarda el dict no sea vacio
      toTakeAout.append(key)
    elif not dictio[key][1]:#se verifica que el valor guardado en la segunda posicion de la lista que guarda el dict no sea vacio
      toTakeAout.append(key)
    elif  "Category" in dictio[key][1]:#verificar si el link  contiene la palabra category lo cual implica que no es una persona
      toTakeAout.append(key)
    elif ":" in dictio[key][1]:#verificar que el link no contenga el caracter : porque no pertenece a una persona
      toTakeAout.append(key)
  for key in toTakeAout:#ciclo para eliminar todos los elementos del dict
     dictio.pop(key)
  return dictio
def filtrar_personas_a(etiquetas,fuenteTexto, analizadorTexto, rutaActual):#funcion para procesar el texto de las etiquetas a y encontrar las personas
    elementos = {}
    text = ""
    for etiqueta in etiquetas:
      elementos.update({etiqueta.text:[etiqueta.text,etiqueta.get("href")]}) #se guarda todos los elementos por los cuales se pasan en un diccionario para facilitar buquedas
    for parrafo in fuenteTexto:
      text = f"{text} {parrafo.text}" #se concatenan todos los textos para que el algoritmo diferenciador funcione
    analizado = analizadorTexto(text)  
    filtrados = {}
    for elemento in analizado.ents:
        if(elemento.label_ == "PERSON" and elemento.text not in rutaActual):#se verifica que el elemento sea una persona y no ese en la ruta por la cual nos encontramos
            filtrados.update({elemento.text:elementos.get(elemento.text)}) # metemos el valor ya validado como persona en el diccionario que retornamos con el nombre de la persona y su link
    filtrados = limpiar_diccionario(filtrados)#limpiamos el diccionario en caso de que el analisis no haya sido potente
    return filtrados

def AumentarConexiones(nuevosNodos, grafo, nodoActual): # funcion donde recibimos el grafo que se instancia en la funcion principal y aumentamos su datosa ingresando los nuevos valores
  for nodo in nuevosNodos:
      if(nodo not in grafo.graph.keys()):
        grafo.add_vertex(nodo)
      if nodo not in grafo.graph[nodoActual]:
        grafo.add_edge(nodoActual, nodo)


import  requests
from bs4 import BeautifulSoup
import spacy

def buscar_ruta(urlInicial=False):#Funcion para buscar la ruta posible entre dos personas
  grafo = Graph()#instanciamos el grafo que se va a retornar
  urlActual =""
  RutaActual = []
  if not urlInicial:
    personaInicial = input("Ingrese P1: ")
    nombreLista= personaInicial.split(" ")
    nombreFormaCorrecta = "_".join(nombreLista)#para buscar una persona en wikipedia los espacios se deben cambiar por _
    urlActual = "https://en.wikipedia.org//wiki/" + nombreFormaCorrecta
    
  else:
    personaUrlInicial = input("Ingrese url de P1: ")
    urlActual = personaUrlInicial
    listaUrl = personaUrlInicial.split("/")
    nombre= listaUrl[-1]
    personaInicial = nombre.replace("_"," ")
  RutaActual.append(personaInicial)#iniciamos la ruta en la persona de inicio
  grafo.add_vertex(personaInicial)
  personaFinal = input("Ingrese P2: ")
  analizadorTexto = spacy.load("en_core_web_sm")
  found = False#iniciamos con el supuesto de que aun no encontramos a la persona final
  personasPasadas = 0#Iniciamos un contador para saber por cuantas personas se pasan
  while not found and personasPasadas<50:
    personasPasadas +=1
    RESPUESTA = requests.get(urlActual)#obtenemos el html de la pagina
    html = RESPUESTA.text
    soup = BeautifulSoup(html, "html.parser")  
    etiquetas_a = soup.find_all("a")#buscamos todas la etiquetas a
    etiquetas_p=soup.find_all("p",limit=20)#limitamos el numero de etiquetas p para disminuir el tiempo de procesamiento
    etiquetas_filtradas_dict = filtrar_personas_a(etiquetas_a,etiquetas_p,analizadorTexto, RutaActual)# inciamos la funcion para recibir el diccionario con las personas y sus links de la etiqueta
    AumentarConexiones(etiquetas_filtradas_dict, grafo, personaInicial)#creamos las nuevas conexiones del grafo
    if personaFinal in etiquetas_filtradas_dict.keys(): # se verifica si la persona que se busca se encuentra en las personas de la pagina web de la persona actual
      RutaActual.append(personaFinal)
      found = True#si se encuentre se declara True la variable found y se agrega a la persona a la ruta como elemento final y se sale del ciclo
      break
    if len(etiquetas_filtradas_dict )>0:# si el diccionario esta vacio significa que la persona en la que se busco ya no tiene mas personas por lo tanto es imposible continuar con la busqueda
        key= next(iter(etiquetas_filtradas_dict)) 
        personaInicial = etiquetas_filtradas_dict.get(key)[0]# se obtiene un valor de la llaves del diccionario y se reasignan los valores de la persona y el link para la nueva busqueda en la pagina de la wiki
        urlActual = "https://en.wikipedia.org/" + etiquetas_filtradas_dict.get(key)[1]
        RutaActual.append(personaInicial) 
    else:
      print("Sin personas en que buscar, interrupcion adelantada")# salida en caso de que ya no sea posibe seguir con el metodo
      break
  if not found:
    print("no están conectados", RutaActual)# si no se encuentra a la persona, se indica que no estan conectados, se imprime la ruta 
  else:
    print("Estan conectados, la ruta fue",RutaActual)#si se ecuentra, se notifica se muetra la ruta
  return grafo,RutaActual# se retorna el grafo formado
grafos = []
rutas= []
for x in range(3):
  grafo,ruta= buscar_ruta()
  grafos.append(grafo)
  rutas.append(ruta)
for i in range(len(rutas)):
  print(rutas[i])
  grafos[i].print_graph()


Ingrese P1: j
Ingrese P2: k
Sin personas en que buscar, interrupcion adelantada
no están conectados ['j', 'jay', 'J.R.R. Tolkien', 'Tolkien', 'Dome Karukoski', 'Pamela Tola', 'Lauri Tilkanen']
